# Data Setup

In [1]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd

### Read in just the beginning of each file to determine what columns are needed

In [23]:
# Function to process a chunk of data
def process_data(chunk, columns=None):
    # If columns is not None, keep only those columns
    if columns is not None:
        chunk = chunk[columns]
    return chunk

# Function to read data in chunks and process each chunk
def load_data(file_name, head = None, columns=None, chunksize = 1000):
    chunks = []
    count = 0
    with gzip.open(file_name) as fin:
        for chunk in pd.read_json(fin, lines=True, chunksize=chunksize):
            # Process the chunk
            processed_chunk = process_data(chunk, columns)
            chunks.append(processed_chunk)
            
            count += 1
            # break if reaches the head-th chunk
            if (head is not None) and (count > head):
                break

    # Combine all chunks into a single DataFrame
    df = pd.concat(chunks, ignore_index=True)
    
    return df

In [19]:
DIR = 'C:\\Users\\jesse\\Desktop\\Honors Project\\goodreads_data\\raw\\'
ya_books = load_data(DIR + 'goodreads_books_young_adult.json.gz', head = 1)
ya_reviews = load_data(DIR + 'goodreads_reviews_young_adult.json.gz', head = 1)
ya_interactions = load_data(DIR + 'goodreads_interactions_young_adult.json.gz', head = 1)
#books_works = load_data(DIR + 'goodreads_book_works.json.gz', head = 1)
#reviews = load_data(DIR + 'goodreads_reviews_spoiler.json.gz', head = 1)

In [13]:
#books_works.columns

In [14]:
#books_works

Books Works IGNORE FOR NOW

In [15]:
ya_reviews

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,2767052,248c011811e945eca861b5c31a549291,5,I cracked and finally picked this up. Very enj...,Wed Jan 13 13:38:25 -0800 2010,Wed Mar 22 11:46:36 -0700 2017,Sun Mar 25 00:00:00 -0700 2012,Fri Mar 23 00:00:00 -0700 2012,24,25
1,7504b2aee1ecb5b2872d3da381c6c91e,23302416,84c0936a0f9868f38e75d2f9a5cb761e,5,I read this book because my fifth grade son wa...,Wed Jan 21 18:40:59 -0800 2015,Wed Oct 26 03:44:13 -0700 2016,,,0,0


Keep from ya_reviews:
['book_id', 'review_id', 'rating', 'review_text', 'n_votes', 'n_comments']

In [16]:
ya_books

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,,1,[147734],US,,"[{'count': '1057', 'name': 'to-read'}, {'count...",B0056A00P4,true,4.04,B0056A00P4,"[519546, 1295074, 21407416]",This is the final tale in the bestselling auth...,,https://www.goodreads.com/book/show/12182387-t...,"[{'author_id': '50873', 'role': ''}, {'author_...",,,,,,,,https://www.goodreads.com/book/show/12182387-t...,https://s.gr-assets.com/assets/nophoto/book/11...,12182387,4,285263,"The Passion (Dark Visions, #3)","The Passion (Dark Visions, #3)"
1,,2,[425995],US,,"[{'count': '1010', 'name': 'to-read'}, {'count...",B006KLYIAG,true,3.80,B006KLYIAG,"[13400912, 13327517, 18107102, 15797097, 11472...",Life should be simple for Cassie.\nFor the sma...,,https://www.goodreads.com/book/show/20135365-h...,"[{'author_id': '5395324', 'role': ''}]",,,,,,,,https://www.goodreads.com/book/show/20135365-h...,https://s.gr-assets.com/assets/nophoto/book/11...,20135365,5,18450480,Hope's Daughter,Hope's Daughter


In [17]:
ya_books.columns

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series'],
      dtype='object')

Keep from ya_books: ['text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'is_ebook', 'average_rating',
       'similar_books', 'description', 'authors',
       'publisher', 'num_pages', 'publication_day',
       'publication_month', 'edition_information', 'publication_year', 'book_id', 'ratings_count', 'title']

In [20]:
ya_interactions

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
0,8842281e1d1347389f2ab93d60773d4d,18667753,be53fe83a6fc83474052f84692f6e90a,False,0,,Wed Mar 29 00:12:52 -0700 2017,Wed Mar 29 00:12:52 -0700 2017,,
1,8842281e1d1347389f2ab93d60773d4d,428263,2030f56879ebcc307a4b9cd8c83200e8,False,0,,Mon Mar 27 22:01:42 -0700 2017,Mon Mar 27 22:01:42 -0700 2017,,


In [21]:
ya_interactions.columns

Index(['user_id', 'book_id', 'review_id', 'is_read', 'rating',
       'review_text_incomplete', 'date_added', 'date_updated', 'read_at',
       'started_at'],
      dtype='object')

ya_interactions keep: ['user_id', 'book_id', 'review_id', 'is_read', 'rating']

### Reading in necessary files with limited columns and reducing numerical columns with smaller dtypes

books

In [166]:
ya_books = load_data(DIR + 'goodreads_books_young_adult.json.gz', columns = ['book_id','title', 'series', 'text_reviews_count','ratings_count','country_code', 'language_code',
       'popular_shelves', 'is_ebook', 'average_rating',
       'similar_books', 'description', 'authors',
       'publisher', 'num_pages', 'publication_day',
       'publication_month',  'publication_year',  'edition_information'])
ya_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93398 entries, 0 to 93397
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   book_id              93398 non-null  int64  
 1   title                93398 non-null  object 
 2   series               93398 non-null  object 
 3   text_reviews_count   93398 non-null  int64  
 4   ratings_count        93398 non-null  int64  
 5   country_code         93398 non-null  object 
 6   language_code        93398 non-null  object 
 7   popular_shelves      93398 non-null  object 
 8   is_ebook             93398 non-null  object 
 9   average_rating       93398 non-null  float64
 10  similar_books        93398 non-null  object 
 11  description          93398 non-null  object 
 12  authors              93398 non-null  object 
 13  publisher            93398 non-null  object 
 14  num_pages            93398 non-null  object 
 15  publication_day      93398 non-null 

In [31]:
ya_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93398 entries, 0 to 93397
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   book_id              93398 non-null  int64  
 1   title                93398 non-null  object 
 2   series               93398 non-null  object 
 3   text_reviews_count   93398 non-null  int64  
 4   ratings_count        93398 non-null  int64  
 5   country_code         93398 non-null  object 
 6   language_code        93398 non-null  object 
 7   popular_shelves      93398 non-null  object 
 8   is_ebook             93398 non-null  object 
 9   average_rating       93398 non-null  float64
 10  similar_books        93398 non-null  object 
 11  description          93398 non-null  object 
 12  authors              93398 non-null  object 
 13  publisher            93398 non-null  object 
 14  num_pages            93398 non-null  object 
 15  publication_day      93398 non-null 

In [136]:
#for int and float types, print out the min and max, followed by the column name
for col in ya_books.columns:
    if ya_books[col].dtype == 'int64' or ya_books[col].dtype == 'float64':
        print(ya_books[col].min(), ya_books[col].max(), col)

50 36524503 book_id
0 142645 text_reviews_count
0 4899965 ratings_count
0.0 5.0 average_rating


Downgrade types (book_id to int32, text_reviews_count to int32, ratings_count to int32, and average_rating to float16)

In [167]:
ya_books['book_id']= ya_books['book_id'].astype('int32')
ya_books['text_reviews_count'] = ya_books['text_reviews_count'].astype('int32')
ya_books['ratings_count'] = ya_books['ratings_count'].astype('int32')
ya_books['average_rating'] = ya_books['average_rating'].astype('float16')

Reviews

In [29]:
ya_reviews = load_data(DIR + 'goodreads_reviews_young_adult.json.gz', columns = ['book_id', 'review_id', 'rating', 'review_text', 'n_votes', 'n_comments'])
ya_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2389900 entries, 0 to 2389899
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   book_id      int64 
 1   review_id    object
 2   rating       int64 
 3   review_text  object
 4   n_votes      int64 
 5   n_comments   int64 
dtypes: int64(4), object(2)
memory usage: 109.4+ MB


In [38]:
#print out the min and max for ya_reviews['book_id'] and rating and n_votes and n_comments
#for int and float types, print out the min and max, followed by the column name
for col in ya_reviews.columns:
    if ya_reviews[col].dtype == 'int64' or ya_reviews[col].dtype == 'float64':
        print(ya_reviews[col].min(), ya_reviews[col].max(), col)

For memory reasons, downgrade to min int fit

In [34]:
ya_reviews['book_id'] = ya_reviews['book_id'].astype('int32')
ya_reviews['rating'] = ya_reviews['rating'].astype('int8')
ya_reviews['n_votes'] = ya_reviews['n_votes'].astype('int16')
ya_reviews['n_comments'] = ya_reviews['n_comments'].astype('int16')

Interactions

In [30]:
ya_interactions = load_data(DIR + 'goodreads_interactions_young_adult.json.gz', columns = ['user_id', 'book_id', 'is_read', 'rating'])
ya_interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34919254 entries, 0 to 34919253
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   book_id  int64 
 2   is_read  bool  
 3   rating   int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 832.5+ MB


In [40]:
#for int and float types, print out the min and max, followed by the column name
for col in ya_interactions.columns:
    if ya_interactions[col].dtype == 'int64' or ya_interactions[col].dtype == 'float64':
        print(ya_interactions[col].min(), ya_interactions[col].max(), col)

50 36524503 book_id
0 5 rating


In [42]:
ya_interactions['book_id'] = ya_interactions['book_id'].astype('int32')
ya_interactions['rating'] = ya_interactions['rating'].astype('int8')

### Doing some arbitrary cleaning on the data

#### checking for duplicates

In [ ]:
#check for duplicate book_id in ya_books
ya_books['book_id'].duplicated().sum()

0

In [ ]:
#check for duplicate book_id, user_id in ya_interactions
ya_interactions[['book_id', 'user_id']].duplicated().sum()

0

In [ ]:
#check for duplicate review_id in ya_reviews
ya_reviews['review_id'].duplicated().sum()

0

In [169]:
#replace blank values with NaN
ya_books.replace('', np.nan, inplace=True)
ya_reviews.replace('', np.nan, inplace=True)
ya_interactions.replace('', np.nan, inplace=True)

In [211]:
def extract_author_ids(author_list):
    try:
        # Extract the 'author_id' from each dictionary and return a list of ids
        return [d['author_id'] for d in author_list if 'author_id' in d]
    except TypeError:
        return None

ya_books['authors'] = ya_books['authors'].apply(extract_author_ids)

ya_books['series'] = ya_books['series'].apply(lambda x: np.nan if len(x)==0 else x)

ya_books

,book_id,title,series,text_reviews_count,ratings_count,country_code,language_code,popular_shelves,is_ebook,average_rating,similar_books,description,authors,publisher,num_pages,publication_day,publication_month,publication_year,edition_information
0,12182387,"The Passion (Dark Visions, #3)",[147734],1,4,US,NaN,"[{'count': '1057', 'name': 'to-read'}, {'count...",true,4.039062,"[519546, 1295074, 21407416]",This is the final tale in the bestselling auth...,"[50873, 232533]",NaN,NaN,NaN,NaN,NaN,NaN
1,20135365,Hope's Daughter,[425995],2,5,US,NaN,"[{'count': '1010', 'name': 'to-read'}, {'count...",true,3.800781,"[13400912, 13327517, 18107102, 15797097, 11472...",Life should be simple for Cassie.\nFor the sma...,[5395324],NaN,NaN,NaN,NaN,NaN,NaN
2,21401181,"Half Bad (Half Life, #1)",[493993],17,33,US,NaN,"[{'count': '1799', 'name': 'fantasy'}, {'count...",true,3.800781,"[15728807, 17182499, 15673520, 16081758, 17842...",Wanted by no one.\nHunted by everyone.\nSixtee...,[7314532],Viking Children's,416,4,3,2014,NaN
3,10099492,Twelfth Grade Kills (The Chronicles of Vladimi...,[176160],9,152,US,eng,"[{'count': '7173', 'name': 'to-read'}, {'count...",true,4.351562,"[25861113, 7430195, 18765937, 6120544, 3247550...",It all comes down to this.\nVlad's running out...,[293603],NaN,NaN,NaN,NaN,NaN,NaN
4,22642971,The Body Electric,NaN,428,1525,US,eng,"[{'count': '9481', 'name': 'to-read'}, {'count...",false,3.710938,"[20499652, 17934493, 13518102, 16210411, 17149...",The future world is at peace.\nElla Shepherd h...,[4018722],Scripturient Books,351,6,10,2014,Special Edition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93393,18221503,"Ölmem Gerekirse (Revenants, #3)",[294897],3,21,US,tur,"[{'count': '2641', 'name': 'to-read'}, {'count...",false,4.210938,"[11413298, 13508069, 7849034, 10552965, 155206...",Sevdigini Kurtarmak Icin Ne Kadarina Hazirsin?...,[3353516],Akil Celen Kitaplar,408,NaN,NaN,2013,NaN
93394,8987191,"The Mockingbirds (The Mockingbirds, #1)",[238215],9,42,US,en-US,"[{'count': '12325', 'name': 'to-read'}, {'coun...",true,3.789062,"[8574517, 6440071, 6625948, 7036227, 6768407, ...",Some schools have honor codes.\nOthers have ha...,[3081700],"Little, Brown Books for Young Readers",360,2,11,2010,NaN
93395,1885730,Joel and Cat Set the Story Straight,NaN,55,1094,US,NaN,"[{'count': '1684', 'name': 'to-read'}, {'count...",false,3.779297,"[1957880, 2490879, 8030321, 13152142, 6849661,...",NaN,"[348478, 570577]",Penguin,240,30,7,2007,NaN
93396,23636536,Another Day,[540527],14,29,US,eng,"[{'count': '2401', 'name': 'to-read'}, {'count...",true,3.669922,"[26802671, 20662291, 23615716, 23870836, 21490...",The eagerly anticipated companion to David Lev...,[11664],Knopf Books for Young Readers,336,25,8,2015,NaN


#### checking for duplicates

In [212]:
#check for duplicate book_id in ya_books
ya_books['book_id'].duplicated().sum()

0

In [213]:
#check for duplicate book_id, user_id in ya_interactions
ya_interactions[['book_id', 'user_id']].duplicated().sum()

0

In [214]:
#check for duplicate review_id in ya_reviews
ya_reviews['review_id'].duplicated().sum()

0

#### ya_books

In [215]:
ya_books[ya_books['series'].str.contains(',', na=False)]

,book_id,title,series,text_reviews_count,ratings_count,country_code,language_code,popular_shelves,is_ebook,average_rating,similar_books,description,authors,publisher,num_pages,publication_day,publication_month,publication_year,edition_information


In [216]:
ya_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93398 entries, 0 to 93397
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   book_id              93398 non-null  int32  
 1   title                93398 non-null  object 
 2   series               54640 non-null  object 
 3   text_reviews_count   93398 non-null  int32  
 4   ratings_count        93398 non-null  int32  
 5   country_code         93398 non-null  object 
 6   language_code        63766 non-null  object 
 7   popular_shelves      93398 non-null  object 
 8   is_ebook             93398 non-null  object 
 9   average_rating       93398 non-null  float16
 10  similar_books        93398 non-null  object 
 11  description          89110 non-null  object 
 12  authors              93398 non-null  object 
 13  publisher            74799 non-null  object 
 14  num_pages            71695 non-null  object 
 15  publication_day      64937 non-null 

Nulls found in series, language_code, description, publisher, num_pages, publication_day, publication_month, publication_year, and edition_information (a lot)

##### ya_books language code

In [217]:
#print out books where language_code is null
ya_books[ya_books['language_code'].isnull()]

,book_id,title,series,text_reviews_count,ratings_count,country_code,language_code,popular_shelves,is_ebook,average_rating,similar_books,description,authors,publisher,num_pages,publication_day,publication_month,publication_year,edition_information
0,12182387,"The Passion (Dark Visions, #3)",[147734],1,4,US,NaN,"[{'count': '1057', 'name': 'to-read'}, {'count...",true,4.039062,"[519546, 1295074, 21407416]",This is the final tale in the bestselling auth...,"[50873, 232533]",NaN,NaN,NaN,NaN,NaN,NaN
1,20135365,Hope's Daughter,[425995],2,5,US,NaN,"[{'count': '1010', 'name': 'to-read'}, {'count...",true,3.800781,"[13400912, 13327517, 18107102, 15797097, 11472...",Life should be simple for Cassie.\nFor the sma...,[5395324],NaN,NaN,NaN,NaN,NaN,NaN
2,21401181,"Half Bad (Half Life, #1)",[493993],17,33,US,NaN,"[{'count': '1799', 'name': 'fantasy'}, {'count...",true,3.800781,"[15728807, 17182499, 15673520, 16081758, 17842...",Wanted by no one.\nHunted by everyone.\nSixtee...,[7314532],Viking Children's,416,4,3,2014,NaN
5,31556136,Like Water,NaN,45,109,US,NaN,"[{'count': '3010', 'name': 'to-read'}, {'count...",false,3.890625,[],A gorgeously written and deeply felt literary ...,[6537142],Balzer + Bray,304,17,10,2017,NaN
7,12182388,"The Possessed (Dark Visions, #2)",[150778],1,4,US,NaN,"[{'count': '1032', 'name': 'to-read'}, {'count...",true,3.919922,"[84066, 1295074]",NaN,"[50873, 232533]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93385,282240,"Demon Witch (The Ravenscliff, #2)","[187755, 457095]",17,491,US,NaN,"[{'count': '248', 'name': 'to-read'}, {'count'...",false,3.990234,"[10136093, 9637544, 18106312, 8533276, 2513197...",Devon March matches wits and wizardry against ...,[164102],ReganBooks,388,1,7,2004,NaN
93386,25332051,The Queen of Bright and Shiny Things,NaN,8,19,US,NaN,"[{'count': '10695', 'name': 'to-read'}, {'coun...",false,3.750000,"[23310742, 22880504, 22011484, 23281781, 27132...",Sage Czinski is trying really hard to be perfe...,[835348],Square Fish,352,5,4,2016,NaN
93389,22521638,The Monster Within,[722367],3,8,US,NaN,"[{'count': '1671', 'name': 'to-read'}, {'count...",true,3.810547,"[17452685, 31925124, 17794299, 18209531, 10726...",The moment seventeen-year-old Samantha Thompso...,[4528793],NaN,NaN,NaN,NaN,NaN,NaN
93390,13536362,"The Voice on the Radio (Janie Johnson, #3)",[152658],20,126,US,NaN,"[{'count': '2703', 'name': 'to-read'}, {'count...",false,3.609375,"[522686, 403651, 1152758, 268918, 936501, 5247...",NaN,[9059],NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
#print number of books in each language_code
ya_books['language_code'].value_counts().head(50)

language_code
eng      35989
en-US     4996
en-GB     2922
ger       2798
spa       2511
fre       1788
ita       1761
ind       1623
nl        1430
por       1156
cze        787
tur        613
swe        609
fin        494
pol        392
dan        383
en-CA      366
bul        332
slo        291
gre        281
hun        276
rum        243
nor        176
per        160
msa        134
scr        122
srp        111
vie        110
est        103
rus        101
heb         91
lav         89
lit         82
cat         72
ara         66
tha         49
jpn         30
zho         24
nob         22
ben         21
kat         21
fil         18
slv         18
en          13
ukr         13
tam         11
nno          6
isl          6
pes          6
mal          5
Name: count, dtype: int64

In [219]:
#print out books where language_code is en or en-CA
ya_books_eng = ya_books[ya_books['language_code'].isin(['en','en-CA','en-GB','en-US','eng'])]

Both en and en-CA appear to be english, as do eng, en-USA, and en-GB. Let's just keep these for now.

In [220]:
ya_books_eng.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44286 entries, 3 to 93397
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   book_id              44286 non-null  int32  
 1   title                44286 non-null  object 
 2   series               28406 non-null  object 
 3   text_reviews_count   44286 non-null  int32  
 4   ratings_count        44286 non-null  int32  
 5   country_code         44286 non-null  object 
 6   language_code        44286 non-null  object 
 7   popular_shelves      44286 non-null  object 
 8   is_ebook             44286 non-null  object 
 9   average_rating       44286 non-null  float16
 10  similar_books        44286 non-null  object 
 11  description          42969 non-null  object 
 12  authors              44286 non-null  object 
 13  publisher            34881 non-null  object 
 14  num_pages            34348 non-null  object 
 15  publication_day      33655 non-null  obje

##### Dropping some uneeded columns

In [221]:
#drop edition_information column not on a copy
ya_books_eng = ya_books_eng.copy()
ya_books_eng = ya_books_eng.drop(['edition_information', 'publication_month', 'publication_day'], axis=1)

In [222]:
ya_books_eng.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44286 entries, 3 to 93397
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   book_id             44286 non-null  int32  
 1   title               44286 non-null  object 
 2   series              28406 non-null  object 
 3   text_reviews_count  44286 non-null  int32  
 4   ratings_count       44286 non-null  int32  
 5   country_code        44286 non-null  object 
 6   language_code       44286 non-null  object 
 7   popular_shelves     44286 non-null  object 
 8   is_ebook            44286 non-null  object 
 9   average_rating      44286 non-null  float16
 10  similar_books       44286 non-null  object 
 11  description         42969 non-null  object 
 12  authors             44286 non-null  object 
 13  publisher           34881 non-null  object 
 14  num_pages           34348 non-null  object 
 15  publication_year    37398 non-null  object 
dtypes: float1

Rest of nulls are in series, description, publisher, num_pages, and publication_year. Will leave as-is for now -- will be handled in system building

#### ya_reviews

In [224]:
ya_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2389900 entries, 0 to 2389899
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   book_id      int32 
 1   review_id    object
 2   rating       int8  
 3   review_text  object
 4   n_votes      int16 
 5   n_comments   int16 
dtypes: int16(2), int32(1), int8(1), object(2)
memory usage: 57.0+ MB


no nulls

In [225]:
ya_interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34919254 entries, 0 to 34919253
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   book_id  int32 
 2   is_read  bool  
 3   rating   int8  
dtypes: bool(1), int32(1), int8(1), object(1)
memory usage: 466.2+ MB


no nulls either

In [226]:
#how many false values are there in ya_interactions['is_read']?
ya_interactions['is_read'].value_counts()

is_read
False    19196505
True     15722749
Name: count, dtype: int64

In [227]:

with gzip.open('ya_books.pkl.gz', 'wb') as f:
    ya_books.to_pickle(f)

with gzip.open('ya_reviews.pkl.gz', 'wb') as f:
    ya_reviews.to_pickle(f)

with gzip.open('ya_interactions.pkl.gz', 'wb') as f:
    ya_interactions.to_pickle(f)

RESUME HERE

In [3]:
with gzip.open('ya_books.pkl.gz', 'rb') as f:
    ya_books = pd.read_pickle(f)

with gzip.open('ya_reviews.pkl.gz', 'rb') as f:
    ya_reviews = pd.read_pickle(f)

with gzip.open('ya_interactions.pkl.gz', 'rb') as f:
    ya_interactions = pd.read_pickle(f)

EOFError: Ran out of input